In [1]:
import pyspark
from pyspark.sql import SparkSession

In [2]:
spark = SparkSession.builder \
    .master("local[*]") \
    .appName('test') \
    .getOrCreate()

23/03/01 20:46:37 WARN Utils: Your hostname, Matts-MacBook-Pro.local resolves to a loopback address: 127.0.0.1; using 192.168.1.158 instead (on interface en0)
23/03/01 20:46:37 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


23/03/01 20:46:38 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [3]:
df_zones = spark.read \
    .option("header", "true") \
    .csv('taxi+_zone_lookup.csv')

df_zones.show()

+----------+-------------+--------------------+------------+
|LocationID|      Borough|                Zone|service_zone|
+----------+-------------+--------------------+------------+
|         1|          EWR|      Newark Airport|         EWR|
|         2|       Queens|         Jamaica Bay|   Boro Zone|
|         3|        Bronx|Allerton/Pelham G...|   Boro Zone|
|         4|    Manhattan|       Alphabet City| Yellow Zone|
|         5|Staten Island|       Arden Heights|   Boro Zone|
|         6|Staten Island|Arrochar/Fort Wad...|   Boro Zone|
|         7|       Queens|             Astoria|   Boro Zone|
|         8|       Queens|        Astoria Park|   Boro Zone|
|         9|       Queens|          Auburndale|   Boro Zone|
|        10|       Queens|        Baisley Park|   Boro Zone|
|        11|     Brooklyn|          Bath Beach|   Boro Zone|
|        12|    Manhattan|        Battery Park| Yellow Zone|
|        13|    Manhattan|   Battery Park City| Yellow Zone|
|        14|     Brookly

In [4]:
df_fhv = spark.read \
    .option("header", "true") \
    .csv('fhvhv_tripdata_2021-06.csv')

In [7]:
df_fhv.show()

+--------------------+-------------------+-------------------+------------+------------+-------+----------------------+
|dispatching_base_num|    pickup_datetime|   dropoff_datetime|PULocationID|DOLocationID|SR_Flag|Affiliated_base_number|
+--------------------+-------------------+-------------------+------------+------------+-------+----------------------+
|              B02764|2021-06-01 00:02:41|2021-06-01 00:07:46|         174|          18|      N|                B02764|
|              B02764|2021-06-01 00:16:16|2021-06-01 00:21:14|          32|         254|      N|                B02764|
|              B02764|2021-06-01 00:27:01|2021-06-01 00:42:11|         240|         127|      N|                B02764|
|              B02764|2021-06-01 00:46:08|2021-06-01 00:53:45|         127|         235|      N|                B02764|
|              B02510|2021-06-01 00:45:42|2021-06-01 01:03:33|         144|         146|      N|                  null|
|              B02510|2021-06-01 00:18:1

In [5]:
df_fhv.repartition(12)

DataFrame[dispatching_base_num: string, pickup_datetime: string, dropoff_datetime: string, PULocationID: string, DOLocationID: string, SR_Flag: string, Affiliated_base_number: string]

In [9]:
df_fhv.write.parquet('fhv_2021_06')

23/02/28 19:51:19 WARN MemoryManager: Total allocation exceeds 95.00% (1,020,054,720 bytes) of heap memory
Scaling row group sizes to 95.00% for 8 writers
23/02/28 19:51:19 WARN MemoryManager: Total allocation exceeds 95.00% (1,020,054,720 bytes) of heap memory
Scaling row group sizes to 84.44% for 9 writers
23/02/28 19:51:19 WARN MemoryManager: Total allocation exceeds 95.00% (1,020,054,720 bytes) of heap memory
Scaling row group sizes to 76.00% for 10 writers
23/02/28 19:51:19 WARN MemoryManager: Total allocation exceeds 95.00% (1,020,054,720 bytes) of heap memory
Scaling row group sizes to 69.09% for 11 writers
23/02/28 19:51:19 WARN MemoryManager: Total allocation exceeds 95.00% (1,020,054,720 bytes) of heap memory
Scaling row group sizes to 63.33% for 12 writers
23/02/28 19:51:19 WARN MemoryManager: Total allocation exceeds 95.00% (1,020,054,720 bytes) of heap memory
Scaling row group sizes to 58.46% for 13 writers
23/02/28 19:51:19 WARN MemoryManager: Total allocation exceeds 95.

In [6]:
df_fhv = spark.read.parquet("fhv_2021_06")

In [7]:
df_fhv

DataFrame[dispatching_base_num: string, pickup_datetime: string, dropoff_datetime: string, PULocationID: string, DOLocationID: string, SR_Flag: string, Affiliated_base_number: string]

In [8]:
df_fhv.printSchema()

root
 |-- dispatching_base_num: string (nullable = true)
 |-- pickup_datetime: string (nullable = true)
 |-- dropoff_datetime: string (nullable = true)
 |-- PULocationID: string (nullable = true)
 |-- DOLocationID: string (nullable = true)
 |-- SR_Flag: string (nullable = true)
 |-- Affiliated_base_number: string (nullable = true)



In [9]:
spark.version

'3.3.2'

In [10]:
from pyspark.sql import functions as F

In [25]:
df_fhv = df_fhv \
.withColumn("pickup_date", F.to_date(df_fhv.pickup_datetime)) \
.withColumn("dropoff_date", F.to_date(df_fhv.dropoff_datetime)) \
.withColumn("duration", F.unix_timestamp(df_fhv.dropoff_datetime) - F.unix_timestamp(df_fhv.pickup_datetime))

In [12]:
df_fhv.filter(df_fhv.pickup_date == '2021-06-15').count()

452470

In [26]:
df_fhv.select("pickup_datetime", "dropoff_datetime", "duration").orderBy("duration", reversed=True).show(5)

+-------------------+-------------------+--------+
|    pickup_datetime|   dropoff_datetime|duration|
+-------------------+-------------------+--------+
|2021-06-06 21:11:14|2021-06-06 21:11:14|       0|
|2021-06-11 12:42:18|2021-06-11 12:42:18|       0|
|2021-06-10 22:04:29|2021-06-10 22:04:29|       0|
|2021-06-04 12:49:13|2021-06-04 12:49:13|       0|
|2021-06-26 23:33:54|2021-06-26 23:33:55|       1|
+-------------------+-------------------+--------+
only showing top 5 rows



In [40]:
df_fhv.registerTempTable('df_fhv')
df_zones.registerTempTable('df_zones')

In [43]:
spark.sql("SELECT Zone from df_zones")

DataFrame[Zone: string]

In [30]:
spark.sql("SELECT max(duration)/3600 from df_fhv").show()

+----------------------+
|(max(duration) / 3600)|
+----------------------+
|      66.8788888888889|
+----------------------+



In [32]:
spark.sql("SELECT max(duration)/3600 from df_fhv where pickup_date = '2021-06-15'").show()

+----------------------+
|(max(duration) / 3600)|
+----------------------+
|     9.402222222222223|
+----------------------+



In [46]:
spark.sql("""SELECT zones.Zone, 
count(fhv.PULocationID) as count, zones.Zone from df_fhv as fhv join df_zones as zones
on fhv.PULocationID = zones.LocationID group by zones.Zone order by -count""").show()

+--------------------+------+--------------------+
|                Zone| count|                Zone|
+--------------------+------+--------------------+
| Crown Heights North|231279| Crown Heights North|
|        East Village|221244|        East Village|
|         JFK Airport|188867|         JFK Airport|
|      Bushwick South|187929|      Bushwick South|
|       East New York|186780|       East New York|
|TriBeCa/Civic Center|164344|TriBeCa/Civic Center|
|   LaGuardia Airport|161596|   LaGuardia Airport|
|            Union Sq|158937|            Union Sq|
|        West Village|154698|        West Village|
|             Astoria|152493|             Astoria|
|     Lower East Side|151020|     Lower East Side|
|        East Chelsea|147673|        East Chelsea|
|Central Harlem North|146402|Central Harlem North|
|Williamsburg (Nor...|143683|Williamsburg (Nor...|
|          Park Slope|143594|          Park Slope|
|  Stuyvesant Heights|141427|  Stuyvesant Heights|
|        Clinton East|139611|  